In [33]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import re

# Keep date
from datetime import date
today = date.today()
today = today.strftime("%d%B%Y")

# BBLAM

In [79]:
url = "https://www.bblam.co.th/products/mutual-funds?asset_class=&region=&risk_level="
browser = webdriver.Firefox(executable_path = r'C:\Users\DELL\geckodriver.exe')
browser.get(url)

detail_list = []
#Accept the website's restriction
accept_point = browser.find_element_by_css_selector("a.btn.btn-accept.cookie-allow")
browser.execute_script("arguments[0].click();", accept_point)
sleep(1)

# Collect every funds excluding the term funds
main_tb = browser.find_element_by_id("maintable")
trs = main_tb.find_elements_by_css_selector("tr")
tr_dict = {}
for tr in trs[1:]:
    if tr.get_attribute("class")=="fund-type":
        key = tr.text
        tr_dict.setdefault(key,[])
    else:
        tr_dict[key].append(tr)

# Ignore term funds and outside funds
remove_funds = ['กองทุนรวมตราสารหนี้แบบมีกำหนดระยะเวลา', 'กองทุนรวมตราสารหนี้ต่างประเทศแบบมีกำหนดระยะเวลา', 'กองทุนรวมอสังหาริมทรัพย์', 'กองทุนรวมโครงสร้างพื้นฐาน', 'กองทรัสต์เพื่อการลงทุนในอสังหาริมทรัพย์']
for remove_fund in remove_funds:
    del tr_dict[remove_fund]

# Get names and detail
for k in tr_dict.keys(): 
    data_detail_list = tr_dict[k]
    for data_detail in data_detail_list:
        code_of_fund,name_of_fund = data_detail.text.split("\n")[0],data_detail.text.split("\n")[1]
        nav = data_detail.text.split("\n")[3]
        alink = data_detail.find_element_by_css_selector("a").get_attribute("href")
        detail_list.append([code_of_fund,name_of_fund,nav,alink])

# Making the dataframe of fund code and name.
Detail_df = pd.DataFrame(detail_list,columns = ["Code","Name","NAV","HTML"])

#Get the detail's data of fund 
policy_list = []
for html in Detail_df["HTML"]:
    browser.get(html)
    policy = browser.find_element_by_css_selector("div.summary-detail")
    policy_list.append(policy.text.split("\n")[1])
# browser.close()

# add the policy of fund
Detail_df["Policy"] = pd.Series(policy_list)

url_name_list = Detail_df["Code"]
holding_data_list = []
# get the proportion of master fund at settrade.com
for url_name in url_name_list:
    browser.get(f"https://www.settrade.com/th/mutualfund/quote/{url_name}/asset-allocation")
    sleep(1)
    table_responsive = browser.find_element_by_xpath("//div[contains(text(),'Top 5 Holdings')]/ancestor::*[2]")
    # get holding and date data from url
    data_date = re.sub("ข้อมูล ณ วันที่","",table_responsive.text.split("\n")[1]).strip()
    holding_asset_list = table_responsive.text.split("\n")[3:]
    holding_asset_list.append(data_date)
    holding_asset_list.append(url_name)
    holding_data_list.append(holding_asset_list)
browser.close()

holding_df = pd.DataFrame(holding_data_list,columns = ["top1","proportion1",
                                                   "top2","proportion2",
                                                   "top3","proportion3",
                                                   "top4","proportion4",
                                                   "top5","proportion5",
                                                   "Update_at","Code"])
# Merging to one dataframe
merged_df = Detail_df.merge(holding_df,on ="Code",how = "left")
print(merged_df.head())
merged_df.to_csv(f"get_BBLAM_data/get_data_from_BBLAM_at_{today}.csv",index = False , encoding = "utf-8-sig")

<bound method NDFrame.head of              Code                                               Name      NAV  \
0      B-TREASURY                            บัวหลวงตราสารหนี้ภาครัฐ  11.5526   
1            B-ST                กองทุนเปิดบัวหลวงตราสารหนี้ระยะสั้น  10.0232   
2          B-TNTV                                       บัวหลวงธนทวี  13.7537   
3          BFIXED                                  บัวหลวงตราสารหนี้  12.9410   
4      B-ENHANCED                                บัวหลวงเพิ่มพูน ***  10.1835   
..            ...                                                ...      ...   
95   B-SI-THAIESG    กองทุนรวมบัวหลวงตราสารภาครัฐไทยเพื่อความยั่งยืน  10.1417   
96  B-TOP-THAIESG            กองทุนรวมบัวหลวงทศพลไทยเพื่อความยั่งยืน   9.6536   
97          BTPVD  บัวหลวงตราสารหนี้ภาครัฐสำหรับกองทุนสำรองเลี้ยงชีพ  10.4104   
98          BFPVD        บัวหลวงตราสารหนี้สำหรับกองทุนสำรองเลี้ยงชีพ  10.5837   
99          BEPVD         บัวหลวงตราสารทุนสำหรับกองทุนสำรองเลี้ยงชีพ   7.4670  

In [78]:
url_name_list = ["B-TREASURY"]
# get the proportion of master fund at settrade.com
for url_name in url_name_list:
    browser.get(f"https://www.settrade.com/th/mutualfund/quote/{url_name}/asset-allocation")
    sleep(1)
    table_responsive = browser.find_element_by_xpath("//div[contains(text(),'Top 5 Holdings')]/ancestor::*[2]")
    data_date = re.sub("ข้อมูล ณ วันที่","",table_responsive.text.split("\n")[1]).strip()
    holding_asset_list = table_responsive.text.split("\n")[3:]
    holding_asset_list.append(data_date)
    

['BANK OF THAILAND (CB24801B)', '8.58', 'BANK OF THAILAND (CB24620A)', '8.42', 'BANK OF THAILAND (CB24627A)', '8.42', 'BANK OF THAILAND (CB24613B)', '7.47', 'BANK OF THAILAND (CB24808A)', '7.33', '1 พ.ค. 2567']


# BCAP

In [40]:
url = "https://www.bcap.co.th/th/mutual-funds"
browser = webdriver.Firefox(executable_path = r'C:\Users\DELL\geckodriver.exe')
browser.get(url)
sleep(1)

#Accept the website's restriction
browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
accept_point = browser.find_element_by_css_selector("a.btn.btn-style.btn-brand.mx-2")
browser.execute_script("arguments[0].click();", accept_point)
sleep(1)

# All of the contents
all_contents = browser.find_element_by_css_selector("div.tab-content")
contents = all_contents.find_elements_by_css_selector("div.pb-5")

contents_list = []

for content in contents:
    # get all rows in table
    rows = content.find_elements_by_xpath("//div[contains(@id,'row-')]")
    
    for row in rows:
        #collecting the emphasize contents
        contents_list.append(row.text.split("\n")[0:4])
        
#create dataframe
df = pd.DataFrame(contents_list,columns = ["Ticket","Type","Name","NAV"])

# exclude the index MSCI (irrelevant) and drop duplicate
df = df[~df["Ticket"].str.contains("MSCI Thailand")]
df.drop_duplicates(subset = ["Ticket"],inplace = True)

# All of BCAP fund's names that will be the next url list
URL_names = df["Ticket"]
fund_detail_list = []
# Open the new browser it get url with end of each fund
for URL_name in URL_names:
    URL_name1 = re.sub(r" ","-",URL_name)
    browser.get(f"https://www.bcap.co.th/mutual-funds/{URL_name1}")
    # get the fund's detail
    detail_domtree = browser.find_element_by_css_selector("div.text-editor-normal.text-white.pt-4")
    # get the top five asset that is the fund are holding.
    module_invest_top_asset = browser.find_element_by_xpath("//div[contains(@class,'moduleInvstTopAsset__Table')]")
    invest_top_5 = module_invest_top_asset.text.split("\n")[2:-2]
    # add investment's detail
    invest_top_5.append(URL_name)
    invest_top_5.append(detail_domtree.text)
    fund_detail_list.append(invest_top_5)

# add the top 5 of investment and the detail of fund
temp_df = pd.DataFrame(fund_detail_list,columns = ["top1","proportion1",
                                                   "top2","proportion2",
                                                   "top3","proportion3",
                                                   "top4","proportion4",
                                                   "top5","proportion5",
                                                   "Ticket","Detail"])
browser.close()
# merge dataframe
df = df.merge(temp_df,left_on = df.index,right_on = temp_df.index,how="left")
print(df.head())
df.to_csv(f"get_BBLAM_data/get_data_from_BCAP_at_{today}.csv",index = False , encoding = "utf-8-sig")

       Ticket                 Type  \
0       BMSCG        กองทุนอีทีเอฟ   
1     BMSCITH        กองทุนอีทีเอฟ   
2     BSET100        กองทุนอีทีเอฟ   
3  BCAP-MONEY    กองทุนรวมตลาดเงิน   
4   BCAP-GFIO  กองทุนรวมตราสารหนี้   

                                                Name      NAV  \
0                   กองทุนเปิด BCAP Mid Small CG ETF   9.2029   
1                  กองทุนเปิด BCAP MSCI Thailand ETF   9.4990   
2                         กองทุนเปิด BCAP SET100 ETF   8.7858   
3                    กองทุนเปิดบีแคป มันนี่ มาร์เก็ต  10.2411   
4  กองทุนเปิดบีแคป โกลบอล ฟิกซ์ อินคัม ออพพอร์ทูน...  10.3823   

                                                top1 proportion1  \
0         บริษัท ศรีตรังแอโกรอินดัสทรี จำกัด (มหาชน)        3.33   
1                          บริษัท ปตท. จำกัด (มหาชน)        8.80   
2  บริษัท เดลต้า อีเลคโทรนิคส์ (ประเทศไทย) จำกัด ...        7.95   
3         พันธบัตรธนาคารแห่งประเทศไทยงวดที่ 24/91/67       15.55   
4         พันธบัตรธนาคารแห่งประเทศไทยงวด

In [37]:
URL_names = ["BCAP-GW75 SSF"]
fund_detail_list = []
# Open the new browser it get url with end of each fund
for URL_name in URL_names:
    URL_name = re.sub(r" ","-",URL_name)
    print(URL_name)
    browser.get(f"https://www.bcap.co.th/mutual-funds/{URL_name}")
    # get the fund's detail
    detail_domtree = browser.find_element_by_css_selector("div.text-editor-normal.text-white.pt-4")
#     print(detail_domtree.text)
    # get the top five asset that is the fund are holding.
    module_invest_top_asset = browser.find_element_by_xpath("//div[contains(@class,'moduleInvstTopAsset__Table')]")
    invest_top_5 = module_invest_top_asset.text.split("\n")[2:-2]
    invest_top_5.append(detail_domtree.text)
    fund_detail_list.append(invest_top_5)
temp = pd.DataFrame(fund_detail_list)

BCAP-GW75-SSF


In [44]:
temp_df[temp_df["Ticket"]==""]

,top1,proportion1,top2,proportion2,top3,proportion3,top4,proportion4,top5,proportion5,Ticket,Detail
